In [6]:
%load_ext autoreload
%autoreload 2

import os,sys
import h5py  
import numpy as np
np.seterr(invalid=['ignore','warn'][0])
import math,cmath
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.titlesize'] = 20
mpl.rcParams['figure.figsize'] = [6.4*1.2,4.8*1.2]
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['lines.marker'] = 's'
mpl.rcParams['lines.linestyle'] = ''
mpl.rcParams['lines.markersize'] = 12
mpl.rcParams['errorbar.capsize'] = 12
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['legend.fontsize'] = 24

mpl.rcParams.update({"axes.grid" : True})
import util as yu

yu.flag_fast=True

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
ensembles=['cA211.530.24','cA2.09.48']

ens2data={}
for ens in ensembles:
    path={
        'cA211.530.24':'/p/project/pines/li47/code/projectData/NST_c/data/cA211.530.24_NST_a_daint.h5',
        'cA2.09.48':'/p/project/pines/li47/code/projectData/NST_c/data/cA2.09.48_NST_c_booster.h5',
        }[ens]
    ens2data[ens]=yu.load(path)
    print('Ncfg='+str(yu.deepKey(ens2data[ens]['2pt'],2).shape[0]))
    print()

lat=None
data=None
Ncfg=None
Tpack=None
    
def updateEns(ens):
    global lat,data,Ncfg,Tpack
    lat=yu.LatticeEnsemble(ens)
    data=ens2data[ens]
    Ncfg=yu.deepKey(data['2pt'],2).shape[0]
    Tpack=yu.TPack[ens]
updateEns(ensembles[0])

loading: /p/project/pines/li47/code/projectData/NST_c/data/cA211.530.24_NST_a_daint.h5
190/190: diags/Z3pt/srcs/Z3pt.h5_NJNpi-a-Nsrc3*3                               
Ncfg=2467

loading: /p/project/pines/li47/code/projectData/NST_c/data/cA2.09.48_NST_c_booster.h5
423/423: diags/Z3pt-pi0f/srcs/Z3pt.h5_NJNpi-a-Nsrc1*4                                
Ncfg=1228



In [8]:
flags={
    'cc2pt':True, # average quantities related by complex conjugation for 2pt
    'cc3pt':True, # same for 3pt (Removal of vacuum expectation value requires 'cc2pt'=='cc3pt')
    'll2pt':True, # average irrep rows 'l1' and 'l2' (Parity breaking effect of tmQCD has been taken care of)
    'll3pt':True, # same for 3pt (This flag has no effect if 'll2pt'=True and spin-projection is done)
    'remove_pi0VEV':True, 
}

def get2pt(opa,opb,diags=yu.diags_all):
    if opa.split(';')[-1]=='12':
        opa1=';'.join(opa.split(';')[:-1]+['n,pi+']); opa2=';'.join(opa.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get2pt(opa1,opb,diags)-np.sqrt(1/3)*get2pt(opa2,opb,diags)
    if opb.split(';')[-1]=='12':
        opb1=';'.join(opb.split(';')[:-1]+['n,pi+']); opb2=';'.join(opb.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get2pt(opa,opb1,diags)-np.sqrt(1/3)*get2pt(opa,opb2,diags)
    res=np.zeros([Ncfg,Tpack],dtype=complex)
    if opa+'_'+opb in data['2pt']:
        t=data['2pt'][opa+'_'+opb]
        res+=np.sum([t[diag] for diag in t.keys() if diag in diags],axis=0)
    if opb+'_'+opa in data['2pt']:
        t=data['2pt'][opb+'_'+opa]; 
        diags_cc={'T', 'T-pi0f'}; 
        res+=np.conj(np.sum([t[diag] for diag in t.keys() if diag in diags_cc and diag in diags],axis=0))
    return res
def get2ptMat(ops,diags=yu.diags_all):
    if flags['ll2pt']:
        flags['ll2pt']=False
        ops_flip=[yu.op_flipl(op) for op in ops]
        t=(get2ptMat(ops,diags=diags)+np.conj(get2ptMat(ops_flip,diags=diags)))/2
        flags['ll2pt']=True
        return t
    t=np.transpose([[get2pt(opa,opb,diags) for opb in ops] for opa in ops],[2,3,0,1])
    if flags['cc2pt']:
        t=(t+np.conj(np.transpose(t,[0,1,3,2])))/2
    return t

def get3pt(opa,opb,insert,diags=yu.diags_all):
    if opa.split(';')[-1]=='12':
        opa1=';'.join(opa.split(';')[:-1]+['n,pi+']); opa2=';'.join(opa.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get3pt(opa1,opb,insert,diags)-np.sqrt(1/3)*get3pt(opa2,opb,insert,diags)
    if opb.split(';')[-1]=='12':
        opb1=';'.join(opb.split(';')[:-1]+['n,pi+']); opb2=';'.join(opb.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get3pt(opa,opb1,insert,diags)-np.sqrt(1/3)*get3pt(opa,opb2,insert,diags)
    res=np.zeros([Ncfg,int(insert.split('_')[-1])+1],dtype=complex)
    if opa+'_'+opb in data['3pt']:
        t=data['3pt'][opa+'_'+opb][insert]
        res+=np.sum([t[diag] for diag in t.keys() if diag in diags],axis=0)
    if opb+'_'+opa in data['3pt']:
        t=data['3pt'][opb+'_'+opa][insert]; 
        diags_cc={'B3pt','W3pt','Z3pt','T-j', 'T-pi0f-j','T-jPf','B3pt-pi0f','W3pt-pi0f','Z3pt-pi0f'}
        if opa+'_'+opb not in data['3pt'] or 'NJN-pi0f' not in data['3pt'][opa+'_'+opb][insert]:
            diags_cc.add('NJN-pi0i')
        t_add=np.zeros([Ncfg,int(insert.split('_')[-1])+1],dtype=complex)+np.sum([t[diag] for diag in t.keys() if diag in diags_cc and diag in diags],axis=0)
        res+=np.conj(t_add[:,::-1])*(yu.gtCj[insert.split('_')[0]])
    return res
def get3ptMat(opas,opbs,insert,diags=yu.diags_all):
    if flags['ll3pt']:
        flags['ll3pt']=False
        opas_flip=[yu.op_flipl(op) for op in opas]; opbs_flip=[yu.op_flipl(op) for op in opbs]
        sgns=np.array([[yu.op_getl_sgn(opa)*yu.op_getl_sgn(opb) for opb in opbs] for opa in opas])
        sgns*=yu.fourCPTstar[insert.split('_')[0]]
        t=(get3ptMat(opas,opbs,insert,diags=diags)+np.conj(get3ptMat(opas_flip,opbs_flip,insert,diags=diags))*sgns[None,None,:,:])/2
        flags['ll3pt']=True
        return t
    t=np.transpose([[get3pt(opa,opb,insert,diags) for opb in opbs] for opa in opas],[2,3,0,1])
    if flags['cc3pt']:
        flags['cc3pt']=False
        tt=get3ptMat(opbs,opas,insert,diags)[:,::-1]*(yu.gtCj[insert.split('_')[0]])
        flags['cc3pt']=True
        t=(t+np.conj(np.transpose(tt,[0,1,3,2])))/2
    return t

def remove_pi0VEV(Ctij,pi0VEV,opas,opbs):
    if not flags['remove_pi0VEV']:
        return Ctij
    
    factor=yu.coeff['g;0,0,1;G1;N1pi0,a;l1;N,pi'][0][1]
    
    ka=None # index for N(n)pi0(0)
    for i,op in enumerate(opas):
        if yu.getNpar(op)==1:
            qa=i # index for N(n)
            continue
        g,pt,irrep,occ,l,fla=op.split(';')
        if 'pi0' in occ:
            ka=i
            factor_a = factor * {'n,pi+':0,'p,pi0':1,'12':-np.sqrt(1/3)}[fla] * {'l1':1,'l2':-1}[l] * pi0VEV
    kb=None
    for i,op in enumerate(opbs):
        if yu.getNpar(op)==1:
            qb=i
            continue
        g,pt,irrep,occ,l,fla=op.split(';')
        if 'pi0' in occ:
            kb=i
            factor_b = factor * {'n,pi+':0,'p,pi0':1,'12':-np.sqrt(1/3)}[fla] * {'l1':1,'l2':-1}[l] * pi0VEV
            
    res=Ctij.copy()
    if ka is not None:
        res[:,ka,:]-=factor_a*Ctij[:,qa,:]
    if kb is not None:
        res[:,:,kb]-=np.conj(factor_b)*Ctij[:,:,qb]
    if ka is not None and kb is not None:
        res[:,ka,kb]+=factor_a*np.conj(factor_b)*Ctij[:,qa,qb]
    
    # print(factor_a,factor_b)
    # print(pi0VEV)
        
    return res

In [ ]:
def run_t0_dependence(pt,ofs,diags=yu.diags_all,title=None,ylim1=None,ylim2=None,ylim3=None,ylim4=None,ylim5=None,label=''):
    fig, axs = yu.getFigAxs(6,5,sharex=True,sharey='row')
    yu.addRowHeader(axs,['cA24: Energy','cA24: E-vectors','cA24: W','cA48: Energy','cA48: E-vectors','cA48: W'])
    yu.addColHeader(axs,['t0=4','tf=10,12,14 v.s. t0','avg tf v.s. t0','t-t0=4','t/t0~2'])
    # title=title if title=='None' else 'pt={}; ofs={}'.format(pt,ofs)
    # fig.suptitle(title)
    
    eV_func=np.abs
    W_func=np.abs
    
    eV_func=xy_func=np.abs
    
    Nop=len(ofs)
    for i_ens,ens in enumerate(ensembles):
        updateEns(ens)
        tmax0=Tpack
        ops=yu.getops(pt,'l1',ofs)
        dat=[get2ptMat(ops,diags),data['VEV']['pi0f']]
        
        irow=i_ens*3+0; icol=0
        tRef=4; tmin=5
        def func(dat):
            t=yu.meanDeep(dat)
            t2pt,tVEVpi0f=t
            t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
            (eVals,eVecs)=yu.GEVP(t2pt,tRef)
            En=np.log(eVals/np.roll(eVals,-1,axis=0)) if tRef>=0 else np.log(eVals)/tRef
            return En.T
        mean,err,cov=yu.jackknife(dat,func)
        for ind in range(1):
            tmax=tmax0-1-3*ind
            tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
            axs[irow,icol].errorbar(np.arange(tmin,tmax)*lat.a,tMean*lat.aInv,tErr*lat.aInv,color='r',fmt=['s','d','o'][ind])
        axs[irow,icol].set_ylim(ylim1)

        irow=i_ens*3+1
        for j in range(1):
            def func(dat):
                t=yu.meanDeep(dat)
                t2pt,tVEVpi0f=t
                t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
                (eVals,eVecs)=yu.GEVP(t2pt,tRef)
                t=eVecs[:,j,:].T
                t=t/t[0,:]
                return eV_func(t)
            mean,err,cov=yu.jackknife(dat,func)
            for ind in range(1,Nop):
                tmax=tmax0
                tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
                axs[irow,icol].errorbar(np.arange(tmin,tmax)*lat.a,tMean,tErr,color='r',fmt=['s','d','o'][ind])
        # axs[irow,icol].set_yscale('log')
        axs[irow,icol].set_ylim([ylim2,ylim3][i_ens])
        # axs[irow,icol].axis('auto')
        
        irow=i_ens*3+2
        for j in range(1):
            def func(dat):
                t=yu.meanDeep(dat)
                t2pt,tVEVpi0f=t
                t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
                (eVals,eVecs)=yu.GEVP(t2pt,tRef)    
                eVecs_inv_tin=np.linalg.inv(eVecs)
                W=1/(eVecs[:,0,0]*eVecs_inv_tin[:,0,0])-1
                return W_func(W)
            mean,err,cov=yu.jackknife(dat,func)
            for ind in range(0,1):
                tmax=tmax0
                tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]

                axs[irow,icol].errorbar(np.arange(tmin,tmax)*lat.a,tMean,tErr,color='r',fmt=['s','d','o'][ind])
        axs[irow,icol].set_ylim([ylim4,ylim5][i_ens])
            
        irow=i_ens*3+0; icol=1
        tfs=[10,12,14]
        colorDic={10:'r',11:'cyan',12:'g',13:'purple',14:'b'}
        for tRef in range(1,10):
            def func(dat):
                t=yu.meanDeep(dat)
                t2pt,tVEVpi0f=t
                t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
                (eVals,eVecs)=yu.GEVP(t2pt,tRef)
                En=np.log(eVals/np.roll(eVals,-1,axis=0)) if tRef>=0 else np.log(eVals)/tRef
                return En.T
            mean,err,cov=yu.jackknife(dat,func)
            for ind in range(1):
                for i_tf,tf in enumerate(tfs):
                    tMean=mean[ind][tf]; tErr=err[ind][tf]
                    axs[irow,icol].errorbar(tRef*lat.a,tMean*lat.aInv,tErr*lat.aInv,color=colorDic[tf],fmt=['s','d','o'][ind])
        axs[irow,icol].set_ylim(ylim1)
        
        irow=i_ens*3+1
        tfs=[10,12,14]
        colorDic={10:'r',11:'cyan',12:'g',13:'purple',14:'b'}
        for tRef in range(1,10):
            def func(dat):
                t=yu.meanDeep(dat)
                t2pt,tVEVpi0f=t
                t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
                (eVals,eVecs)=yu.GEVP(t2pt,tRef)
                t=eVecs[:,j,:].T
                t=t/t[0,:]
                return eV_func(t)
            mean,err,cov=yu.jackknife(dat,func)
            for ind in range(1,Nop):
                for i_tf,tf in enumerate(tfs):
                    tMean=mean[ind][tf]; tErr=err[ind][tf]
                    axs[irow,icol].errorbar(tRef*lat.a,tMean,tErr,color=colorDic[tf],fmt=['s','d','o'][ind])
        # axs[irow,icol].set_yscale('log')
        axs[irow,icol].set_ylim([ylim2,ylim3][i_ens])
        
        irow=i_ens*3+2
        tfs=[10,12,14]
        colorDic={10:'r',11:'cyan',12:'g',13:'purple',14:'b'}
        for tRef in range(1,10):
            def func(dat):
                t=yu.meanDeep(dat)
                t2pt,tVEVpi0f=t
                t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
                (eVals,eVecs)=yu.GEVP(t2pt,tRef)
                eVecs_inv_tin=np.linalg.inv(eVecs)
                W=1/(eVecs[:,0,0]*eVecs_inv_tin[:,0,0])-1
                return W_func(W)
            mean,err,cov=yu.jackknife(dat,func)
            for ind in range(0,1):
                for i_tf,tf in enumerate(tfs):
                    tMean=mean[ind][tf]; tErr=err[ind][tf]
                    axs[irow,icol].errorbar(tRef*lat.a,tMean,tErr,color=colorDic[tf],fmt=['s','d','o'][ind])
        axs[irow,icol].set_ylim([ylim4,ylim5][i_ens])
        
        irow=i_ens*3+0; icol=2
        
        irow=i_ens*3+1
        tfs=[10,11,12,13,14]
        # colorDic={10:'r',11:'cyan',12:'g',13:'purple',14:'b'}
        for tRef in range(1,10):
            def func(dat):
                t=yu.meanDeep(dat)
                t2pt,tVEVpi0f=t
                t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
                (eVals,eVecs)=yu.GEVP(t2pt,tRef)
                t=eVecs[:,j,:].T
                t=t/t[0,:]
                return eV_func(np.mean(t[:,tfs],axis=1))
            mean,err,cov=yu.jackknife(dat,func)
            for ind in range(1,Nop):
                tMean=mean[0][ind]; tErr=err[0][ind]
                axs[irow,icol].errorbar(tRef*lat.a,tMean,tErr,color='r',fmt=['s','d','o'][ind])
        # axs[irow,icol].set_yscale('log')
        axs[irow,icol].set_ylim([ylim2,ylim3][i_ens])
        # axs[irow,icol].axis('auto')
        
        irow=i_ens*3+2
        tfs=[10,11,12,13,14]
        # colorDic={10:'r',11:'cyan',12:'g',13:'purple',14:'b'}
        for tRef in range(1,10):
            def func(dat):
                t=yu.meanDeep(dat)
                t2pt,tVEVpi0f=t
                t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
                (eVals,eVecs)=yu.GEVP(t2pt,tRef)
                eVecs_inv_tin=np.linalg.inv(eVecs)
                W=1/(eVecs[:,0,0]*eVecs_inv_tin[:,0,0])-1
                return np.mean(W_func(W)[tfs],axis=0)
            mean,err,cov=yu.jackknife(dat,func)
            for ind in range(0,1):
                tMean=mean[0][ind]; tErr=err[0][ind]
                axs[irow,icol].errorbar(tRef*lat.a,tMean,tErr,color='r',fmt=['s','d','o'][ind])
        axs[irow,icol].set_ylim([ylim4,ylim5][i_ens])
        
        irow=i_ens*3+1; icol=3
        for j in range(1):
            def func(dat):
                t=yu.meanDeep(dat)
                t2pt,tVEVpi0f=t
                t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
                (eVals,eVecs)=yu.GEVP(t2pt,-4)
                t=eVecs[:,j,:].T
                t=t/t[0,:]
                return eV_func(t)
            mean,err,cov=yu.jackknife(dat,func)
            for ind in range(1,Nop):
                tmax=tmax0
                tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
                axs[irow,icol].errorbar(np.arange(tmin,tmax)*lat.a,tMean,tErr,color='r',fmt=['s','d','o'][ind])
        # axs[irow,icol].set_yscale('log')
        axs[irow,icol].set_ylim([ylim2,ylim3][i_ens])
        # axs[irow,icol].axis('auto')
        
        irow=i_ens*3+2
        for j in range(1):
            def func(dat):
                t=yu.meanDeep(dat)
                t2pt,tVEVpi0f=t
                t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
                (eVals,eVecs)=yu.GEVP(t2pt,-4)
                eVecs_inv_tin=np.linalg.inv(eVecs)
                W=1/(eVecs[:,0,0]*eVecs_inv_tin[:,0,0])-1
                return W_func(W)
            mean,err,cov=yu.jackknife(dat,func)
            for ind in range(0,1):
                tmax=tmax0
                tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
                axs[irow,icol].errorbar(np.arange(tmin,tmax)*lat.a,tMean,tErr,color='r',fmt=['s','d','o'][ind])
        axs[irow,icol].set_ylim([ylim4,ylim5][i_ens])

        
        irow=i_ens*3+1; icol=4
        tmin=2
        for j in range(1):
            def func(dat):
                t=yu.meanDeep(dat)
                t2pt,tVEVpi0f=t
                t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
                (eVals,eVecs)=yu.GEVP(t2pt,'t/2')
                t=eVecs[:,j,:].T
                t=t/t[0,:]
                return eV_func(t)
            mean,err,cov=yu.jackknife(dat,func)
            for ind in range(1,Nop):
                tmax=tmax0
                tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
                axs[irow,icol].errorbar(np.arange(tmin,tmax)*lat.a,tMean,tErr,color='r',fmt=['s','d','o'][ind])
        # axs[irow,icol].set_yscale('log')
        axs[irow,icol].set_ylim([ylim2,ylim3][i_ens])
        # axs[irow,icol].axis('auto')
        
        irow=i_ens*3+2
        for j in range(1):
            def func(dat):
                t=yu.meanDeep(dat)
                t2pt,tVEVpi0f=t
                t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
                (eVals,eVecs)=yu.GEVP(t2pt,'t/2')
                eVecs_inv_tin=np.linalg.inv(eVecs)
                W=1/(eVecs[:,0,0]*eVecs_inv_tin[:,0,0])-1
                return W_func(W)
            mean,err,cov=yu.jackknife(dat,func)
            for ind in range(0,1):
                tmax=tmax0
                tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
                axs[irow,icol].errorbar(np.arange(tmin,tmax)*lat.a,tMean,tErr,color='r',fmt=['s','d','o'][ind])
        axs[irow,icol].set_ylim([ylim4,ylim5][i_ens])
        
                
    plt.tight_layout()
    plt.savefig('fig/t0_dependence'+label+'.pdf')
    plt.close()
    print(label)
                  
# run_t0_dependence('0,0,0',[('a','p'),('N1pi1,a','12'),],ylim2=[0,0.5],ylim3=[0,0.05],ylim4=[0,0.4],ylim5=[0,0.2],label='_0_N1pi1')
# run_t0_dependence('0,0,1',[('a','p'),('N1pi0,a','12'),],ylim2=[0,0.03],ylim3=[0,0.01],ylim4=[0,0.01],ylim5=[0,0.01],label='_1_N1pi0')
# run_t0_dependence('0,0,1',[('a','p'),('N1pi0,a','12'),],ylim2=[0,0.1],ylim3=[0,0.025],ylim4=[0,0.07],ylim5=[0,0.04],label='_1_N1pi0_sameScale')
# run_t0_dependence('0,0,1',[('a','p'),('N0pi1,a','12'),],ylim2=[0,0.1],ylim3=[0,0.025],ylim4=[0,0.07],ylim5=[0,0.04],label='_1_N0pi1')
# run_t0_dependence('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),],ylim2=[0,0.1],ylim3=[0,0.025],ylim4=[0,0.07],ylim5=[0,0.04],label='_1_N1pi0_N0pi1')

In [ ]:
flags['dt2pt']=False
flags['dt3pt']=True

tfs=[10,12,14]
def get_dt(eVecsa_ltni,eVecsb_ltni):
    eVecsa_ltni=yu.renormalize_eVecs(eVecsa_ltni)
    # eVecsa_ltni=eVecsa_ltni/np.diagonal(eVecsa_ltni,axis1=-2,axis2=-1)[:,:,:,None]
    eVecsa_lni=np.mean(eVecsa_ltni[:,10:14+1],axis=1)
    eVecsa_inv_lin=np.linalg.inv(eVecsa_lni)
    Wa_l=1/(eVecsa_lni[:,0,0]*eVecsa_inv_lin[:,0,0])-1
    eVeca_li=eVecsa_lni[:,0]
    dt2pta_lij=np.einsum('li,lj->lij',np.conj(eVeca_li),eVeca_li)
    if flags['dt2pt']:
        dt2pta_lij[:,1:,1:]*=0
        dt2pta_lij[:,0,0]*=1-np.conj(Wa_l)*Wa_l
        dt2pta_lij[:,1:,0]*=1+Wa_l[:,None]
        dt2pta_lij[:,0,1:]*=1+np.conj(Wa_l[:,None])

    eVecsb_ltni=yu.renormalize_eVecs(eVecsb_ltni)
    # eVecsb_ltni=eVecsb_ltni/np.diagonal(eVecsb_ltni,axis1=-2,axis2=-1)[:,:,:,None]
    eVecsb_lni=np.mean(eVecsb_ltni[:,10:14+1],axis=1)
    eVecsb_inv_lin=np.linalg.inv(eVecsb_lni)
    Wb_l=1/(eVecsb_lni[:,0,0]*eVecsb_inv_lin[:,0,0])-1
    eVecb_li=eVecsb_lni[:,0]
    dt2ptb_lij=np.einsum('li,lj->lij',np.conj(eVecb_li),eVecb_li)
    if flags['dt2pt']:
        dt2ptb_lij[:,1:,1:]*=0
        dt2ptb_lij[:,0,0]*=1-np.conj(Wb_l)*Wb_l
        dt2ptb_lij[:,1:,0]*=1+Wb_l[:,None]
        dt2ptb_lij[:,0,1:]*=1+np.conj(Wb_l[:,None])

    dt3pt_ijlm=np.einsum('li,mj->ijlm',np.conj(eVeca_li),eVecb_li)
    dt3pt_ijlm[1:,1:]*=0
    if flags['dt3pt']:
        d00_lm=1-np.einsum('l,m->lm',np.conj(Wa_l),Wb_l)
        di0_lm=1+np.einsum('l,m->lm',[1,1],Wb_l)
        d0j_lm=1+np.einsum('l,m->lm',np.conj(Wa_l),[1,1])
        dt3pt_ijlm[1:,1:]*=0
        dt3pt_ijlm[0,0]*=d00_lm
        dt3pt_ijlm[1:,0]*=di0_lm[None,:,:]
        dt3pt_ijlm[0,1:]*=d0j_lm[None,:,:]

    return (dt2pta_lij,dt2ptb_lij,dt3pt_ijlm)

def runFF(ptofas,ptofbs,FF,tRefs=[1],label='',ylim1=None):
    fig, axs = yu.getFigAxs(4,len(tRefs),sharex=True,sharey=True if ylim1==None else False)
    yu.addRowHeader(axs,['cA24: noJL','cA24: all','cA48: noJL','cA48: all'])
    yu.addColHeader(axs,['tRef={}'.format(tRef) for tRef in tRefs])
    # title=title if title=='None' else 'opa={}; opb={}; insert={}; prefactor={}'.format(opa,opb,insert,prefactor)
    # fig.suptitle(title)
    
    for i_ens,ens in enumerate(ensembles):
        updateEns(ens)

        mN=lat.getaEN(0); EN=lat.getaEN(1); qk=(2*math.pi)/(lat.L/lat.a); K=np.sqrt(2*mN**2/(EN*(EN+mN)))
        t_Fpi=92.9/lat.aInv
        t_aQ2=(qk**2-(EN-mN)**2)
        t_P=(lat.ampi**2+t_aQ2)/(92.9/lat.aInv*lat.ampi**2)*(t_aQ2)/(4*mN)
        t_Pt=(2*mN**2)/(K*qk*(mN-EN))*t_P; t_Pt2=(2*mN)/(mN-EN)*t_P
        t_Pi=-1j*(2*mN**2)/(K*qk**2)*t_P; t_Pi2=-(2*mN*(EN+mN))/(qk**2)*t_P
        
        t_A2pNN_0=mN/t_Fpi
        t_A2pNN_1=mN/t_Fpi*(lat.ampi**2+t_aQ2)/(lat.ampi**2)
        t_52pNN_1=(lat.amu/lat.ZP)*(lat.ampi**2+t_aQ2)/(lat.ampi**2)/t_Fpi
        t_P2pNN_1=(t_aQ2)/(4*mN**2)*t_A2pNN_1
        
        t_GAzz_00=(-1)*(2*mN*(mN+mN))/(2j*((mN+mN)**2)) * lat.ZA; t_GAzzb_00=t_GAzz_00 * t_A2pNN_0
        t_GAtz_11=(-1)*(2*EN*(EN+mN))/(-4*(EN+mN)*qk) * lat.ZA; t_GAtzb_11=t_GAtz_11 * t_A2pNN_0
        t_GAzz_11=(-1)*(2*EN*(EN+mN))/(2j*((EN+mN)**2+qk**2)) * lat.ZA; t_GAzzb_11=t_GAzz_11 * t_A2pNN_0
        t_GAxx_11=(-1)*(2*EN*(EN+mN))/(2j*((EN+mN)**2)) * lat.ZA; t_GAxxb_11=t_GAxx_11 * t_A2pNN_0
        
        t_GA_01=(-1)*(-1j)*np.sqrt(EN/(2*(EN+mN))) * lat.ZA
        t_G5_01=(-1*np.sqrt(EN*(EN+mN)/2))/qk * lat.ZS; t_G5b_01=t_G5_01 * t_52pNN_1
        t_GP_01_tz=(-1)*np.sqrt(2*EN*(EN+mN))*mN/(qk*(EN-mN)) * lat.ZA; t_GP_01_tz_A=-(2*mN)/(EN-mN)
        t_GP_01_zz=(-1)*1j*np.sqrt(2*EN*(EN+mN))*mN/(qk**2) * lat.ZA; t_GP_01_zz_A=-(2*mN)/(EN-mN)

        t_PCAC_G5=(lat.amu/lat.ZP)/mN
        t_PCAC_GP=t_aQ2/(4*mN**2)

        xUnit=lat.a
        
        pips={
            'GS+_00':[(lat.ZP,'id_j+','P0')],
            'GA_00_zz':[(t_GAzz_00,'g5gz_j-','Pz')],
            'J5_11':[(lat.ZS,'g5_j-','Pz')],
            'GA_11_tz':[(t_GAtz_11,'g5gt_j-','Pz')],
            'GA_11_zz':[(t_GAzz_11,'g5gz_j-','Pz')],
            'GA_11_xx':[(t_GAxx_11*(1/2),'g5gx_j-','Px'),(t_GAxx_11*(1/2),'g5gy_j-','Py')],
            
            'G5b_01':[(t_G5b_01,'g5_j-','Pz')],
            'GA_01':[(t_GA_01*(1/2),'g5gx_j-','Px'),(t_GA_01*(1/2),'g5gy_j-','Py')],
            'GPb_01_tz':[(t_GA_01*(-1/2)*t_GP_01_tz_A*t_P2pNN_1,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_P2pNN_1,'g5gy_j-','Py'),(t_GP_01_tz*t_P2pNN_1,'g5gt_j-','Pz')],
            'GPb_01_zz':[(t_GA_01*(-1/2)*t_GP_01_zz_A*t_P2pNN_1,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_P2pNN_1,'g5gy_j-','Py'),(t_GP_01_zz*t_P2pNN_1,'g5gz_j-','Pz')],
            
            'PCAC_1':[((-1)*(-(EN-mN)),'g5gt_j-','Pz'),((-1)*(1j*qk),'g5gz_j-','Pz'),((-1)*2*lat.amu,'g5_j-','Pz')],
            'PCAC_1_re':[((-1)*(-(EN-mN))*lat.ZA,'g5gt_j-','Pz'),((-1)*(1j*qk)*lat.ZA,'g5gz_j-','Pz'),((-1)*2*lat.amu/lat.ZP*lat.ZS,'g5_j-','Pz')],
            
            'rPCAC_up_1_Ptz':[(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_tz*t_PCAC_GP,'g5gt_j-','Pz')],
            'rPCAC_up_1_Pzz':[(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_zz*t_PCAC_GP,'g5gz_j-','Pz')],
        
            'rPCAC_1_Ptz':[(t_GA_01*(1/2),'g5gx_j-','Px'),(t_GA_01*(1/2),'g5gy_j-','Py'),(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_tz*t_PCAC_GP,'g5gt_j-','Pz')],
            'rPCAC_1_Pzz':[(t_GA_01*(1/2),'g5gx_j-','Px'),(t_GA_01*(1/2),'g5gy_j-','Py'),(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_zz*t_PCAC_GP,'g5gz_j-','Pz')],
        }[FF]
        
        pta,ofas=ptofas; ptb,ofbs=ptofbs

        proj2mat={
            'P0':np.array([[1,0],[0,1]]),
            'Px':np.array([[0,1],[1,0]]),
            'Py':np.array([[0,-1j],[1j,0]]),
            'Pz':np.array([[1,0],[0,-1]]),
        }
        for i_diags,diags in [(0,yu.diags_jLoopless),(1,yu.diags_all)]:
            irow=i_ens*2+i_diags
            for icol,tRef in enumerate(tRefs):
                needsVEV= pta==ptb and 'N-j' in diags and FF in ['GS+_00','J5_11']
                for case in ['noGEVP','GEVP']:
                    pta,ofas=ptofas; ptb,ofbs=ptofbs
                    if case=='noGEVP':
                        ofas=[ofas[0]]; ofbs=[ofbs[0]]
                    
                    def t_get(tf,t_insert):
                        t=[[get3ptMat(yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs),t_insert+'_'+str(tf),diags)[:,1:tf]\
                            for lb in ['l1','l2']] for la in ['l1','l2']]
                        return np.transpose(t,[2,3,4,5,0,1]) # cfg,tc,opa,opb,la,lb
                    
                    dat3pt=[t_get(tf,insert) for prefactor,insert,proj in pips for tf in tfs]
                    dat2pt=[get2ptMat(yu.getops(pta,'l1',ofas),yu.diags_all),get2ptMat(yu.getops(pta,'l2',ofas),diags),
                            get2ptMat(yu.getops(ptb,'l1',ofbs),yu.diags_all),get2ptMat(yu.getops(ptb,'l2',ofbs),diags),]
                    datVEVj=[data['VEV']['j']['id_j+' if FF in ['GS+_00'] else 'g5_j-']]
                    datVEVpi0f=[data['VEV']['pi0f']]
                    dat=[dat3pt,dat2pt,datVEVj,datVEVpi0f]
                    
                    AInv=yu.getCoeMat(pta); BInv=yu.getCoeMat(ptb)

                    def func(dat):
                        t=yu.meanDeep(dat)
                        t3pt,t2pt,tVEVj,datVEVpi0f=t
                        
                        t2pt[0]=remove_pi0VEV(t2pt[0],datVEVpi0f[0],yu.getops(pta,'l1',ofas),yu.getops(pta,'l1',ofas))
                        t2pt[1]=remove_pi0VEV(t2pt[1],datVEVpi0f[0],yu.getops(pta,'l2',ofas),yu.getops(pta,'l2',ofas))
                        t2pt[2]=remove_pi0VEV(t2pt[2],datVEVpi0f[0],yu.getops(ptb,'l1',ofbs),yu.getops(ptb,'l1',ofbs))
                        t2pt[3]=remove_pi0VEV(t2pt[3],datVEVpi0f[0],yu.getops(ptb,'l2',ofbs),yu.getops(ptb,'l2',ofbs))
                        
                        for ind in range(len(t3pt)):
                            for ia,la in enumerate(['l1','l2']):
                                for ib,lb in enumerate(['l1','l2']): 
                                    t3pt[ind][:,:,:,ia,ib]=remove_pi0VEV(t3pt[ind][:,:,:,ia,ib],datVEVpi0f[0],yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs))
                        
                        (eVals_al1,eVecs_al1)=yu.GEVP(t2pt[0],tRef)
                        (eVals_al2,eVecs_al2)=yu.GEVP(t2pt[1],tRef)
                        (eVals_bl1,eVecs_bl1)=yu.GEVP(t2pt[2],tRef)
                        (eVals_bl2,eVecs_bl2)=yu.GEVP(t2pt[3],tRef)
                        
                        eVals_a=(eVals_al1+eVals_al2)/2
                        Ea_tn=np.log(eVals_a/np.roll(eVals_a,-1,axis=0))
                        eVals_b=(eVals_bl1+eVals_bl2)/2
                        Eb_tn=np.log(eVals_b/np.roll(eVals_b,-1,axis=0))

                        eVecsa_ltni=np.array([eVecs_al1,eVecs_al2])
                        eVecsb_ltni=np.array([eVecs_bl1,eVecs_bl2])
                        dt2pta_lij,dt2ptb_lij,dt3pt_ijlm=get_dt(eVecsa_ltni,eVecsb_ltni)
                        
                        t2pta_ltij=np.array([t2pt[0],t2pt[1]])
                        t2pta_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2pta_ltij,dt2pta_lij)),axis=0)
                        
                        t2ptb_ltij=np.array([t2pt[2],t2pt[3]])
                        t2ptb_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2ptb_ltij,dt2ptb_lij)),axis=0)
                        
                        if needsVEV:
                            proj=proj2mat[pips[0][2]]
                            t2pt_sub=np.einsum('tij,ij->t',t2pta_ltij[0],dt3pt_ijlm[:,:,0,0])*proj[0,0]+\
                                np.einsum('tij,ij->t',t2pta_ltij[1],dt3pt_ijlm[:,:,1,1])*proj[1,1]
                        
                        if FF not in ['rPCAC_1_Ptz','rPCAC_1_Pzz']:
                            def t_get2(i):
                                res=0
                                for i_pip,pip in enumerate(pips):
                                    prefactor,insert,proj=pip
                                    proj=proj2mat[proj]
                                    proj2=np.einsum('ba,al,bm->lm',proj,AInv,np.conj(BInv))
                                    res+=prefactor*np.einsum('tijlm,lm,ijlm->t',t3pt[i_pip*len(tfs)+i],proj2,dt3pt_ijlm) # ij for ops; l,m for la,lb; ab for diracs
                                return res
                            t=[(
                                t_get2(i)
                                -(t2pt_sub[tf]*tVEVj[0]*pips[0][0] if needsVEV else 0)
                            )/np.sqrt(
                                t2pta_t[tf]*t2ptb_t[tf]*\
                                # t2pta_t[1:tf][::-1]/t2pta_t[1:tf]*\
                                # t2ptb_t[1:tf]/t2ptb_t[1:tf][::-1]
                                np.exp(-(Ea_tn[tf,0]-Eb_tn[tf,0])*np.array([tf-2*tc for tc in range(1,tf)]))
                            )
                            for i,tf in enumerate(tfs)]
                        else:
                            def t_get2_N(i):
                                res=0
                                for i_pip,pip in enumerate(pips):
                                    if i_pip<2:
                                        continue
                                    prefactor,insert,proj=pip
                                    proj=proj2mat[proj]
                                    proj2=np.einsum('ba,al,bm->lm',proj,AInv,np.conj(BInv))
                                    res+=prefactor*np.einsum('tijlm,lm,ijlm->t',t3pt[i_pip*len(tfs)+i],proj2,dt3pt_ijlm) # ij for ops; l,m for la,lb; ab for diracs
                                return res
                            def t_get2_D(i):
                                res=0
                                for i_pip,pip in enumerate(pips):
                                    if i_pip>=2:
                                        continue
                                    prefactor,insert,proj=pip
                                    proj=proj2mat[proj]
                                    proj2=np.einsum('ba,al,bm->lm',proj,AInv,np.conj(BInv))
                                    res+=prefactor*np.einsum('tijlm,lm,ijlm->t',t3pt[i_pip*len(tfs)+i],proj2,dt3pt_ijlm) # ij for ops; l,m for la,lb; ab for diracs
                                return res

                            t=[(
                                t_get2_N(i)/t_get2_D(i)
                            )
                            for i,tf in enumerate(tfs)]
                    
                        t=yu.prefactorDeep(t,1)
                        return t
                    
                    (mean,err,cov)=yu.jackknife(dat,func)
                    for i_tf,tf in enumerate(tfs):
                        tMean=mean[i_tf];tErr=err[i_tf]
                        axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)*xUnit,tMean,tErr,color='b' if case=='noGEVP' else 'r',fmt=['s','d','o'][i_tf],mfc='white' if case=='noGEVP' else None)                               
                             
    
    plt.tight_layout()
    plt.savefig('fig/'+FF+label+'.pdf')
    plt.close()
    print(FF)
    
# tRefs=[1,2,3,4,5,6,7]
# for FF in ['J5_11']:
#     runFF(('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')]),FF,tRefs=tRefs,label='_')
    
# tRefs=[1,2,3,4,5,6,7]
# for FF in ['GS+_00','GA_00_zz']:
#     runFF(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,0',[('a','p'),('N1pi1,a','12')]),FF,tRefs=tRefs)
# for FF in ['J5_11','GA_11_tz','GA_11_zz','GA_11_xx']:
#     runFF(('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),FF,tRefs=tRefs)
# for FF in ['G5b_01','GA_01','GPb_01_tz','GPb_01_zz']:
#     runFF(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),FF,tRefs=tRefs)
# for FF in ['rPCAC_1_Ptz','rPCAC_1_Pzz']:
#     runFF(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),FF,tRefs=tRefs)

In [9]:
choices={
    # 'En_0':[[(1,t) for t in range(14,14+1)], #cA24
    #         [(1,t) for t in range(16,16+1)]], #cA48
    # 'En_1':[[(1,t) for t in range(11,14+1)],
    #         [(1,t) for t in range(11,14+1)]],
    
    'eV_0':[[(t-4,t) for t in range(10,11+1)],
            [(t-4,t) for t in range(12,13+1)]],
    'eV_1':[[(t-3,t) for t in range(11,12+1)],
            [(t-1,t) for t in range(6,9+1)]],
}


dts=range(1,11)
def run_t0_dep2(pt,ofs,diags=yu.diags_all,ylim1=None,ylim2=None,ylim3=None,ylim4=None,ylim5=None,label=None,choice=None):
    Nop=len(ofs)
    
    fig, axs = yu.getFigAxs(3*2,len(dts),sharex=True,sharey='row')
    yu.addRowHeader(axs,['cA24: Energy','cA24: E-vectors','cA24: W','cA48: Energy','cA48: E-vectors','cA48: W'])
    yu.addColHeader(axs,['dt='+str(dt) for dt in dts])
    
    eV_func=np.abs
    W_func=np.abs
    for i_ens,ens in enumerate(ensembles):
        updateEns(ens)
        ops=yu.getops(pt,'l1',ofs)
        dat=[get2ptMat(ops,diags),data['VEV']['pi0f']]
        
        for i_dt,dt in enumerate(dts):
            irow=i_ens*3+1; icol=i_dt
            def func(dat):
                t=yu.meanDeep(dat)
                t2pt,tVEVpi0f=t
                t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
                (eVals,eVecs)=yu.GEVP(t2pt,-dt,tList=None if dt>=4 else range(Tpack-3))
                t=eVecs[:,0,:].T
                t=t/t[0,:]
                return eV_func(t)
            mean,err,cov=yu.jackknife(dat,func)
            for ind in range(1,Nop):
                tmin=dt+1; tmax=len(mean[ind])
                tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
                axs[irow,icol].errorbar(np.arange(tmin,tmax)*lat.a,tMean,tErr,color='r',fmt=['s','d','o'][ind])
                if choice!=None:
                    for t in range(tmin,tmax):
                        if (t-dt,t) in choices[choice][i_ens]:
                            axs[irow,icol].errorbar(t*lat.a,mean[ind][t],err[ind][t],color='b',fmt=['s','d','o'][ind])
            axs[irow,icol].set_ylim([ylim2,ylim3][i_ens])
            
            irow=i_ens*3+2
            def func(dat):
                t=yu.meanDeep(dat)
                t2pt,tVEVpi0f=t
                t2pt=remove_pi0VEV(t2pt,tVEVpi0f,ops,ops)
                (eVals,eVecs)=yu.GEVP(t2pt,-dt,tList=None if dt>=4 else range(Tpack-3))
                eVecs_inv_tin=np.linalg.inv(eVecs)
                W=1/(eVecs[:,0,0]*eVecs_inv_tin[:,0,0])-1
                return W_func(W)
            mean,err,cov=yu.jackknife(dat,func)
            for ind in range(0,1):
                tmin=dt+1; tmax=len(mean[ind])
                tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
                axs[irow,icol].errorbar(np.arange(tmin,tmax)*lat.a,tMean,tErr,color='r',fmt=['s','d','o'][ind])
                if choice!=None:
                    for t in range(tmin,tmax):
                        if (t-dt,t) in choices[choice][i_ens]:
                            axs[irow,icol].errorbar(t*lat.a,mean[ind][t],err[ind][t],color='b',fmt=['s','d','o'][ind])
            axs[irow,icol].set_ylim([ylim4,ylim5][i_ens])

            
    plt.tight_layout()
    plt.savefig('fig/t0_dep2'+label+'.pdf')
    plt.close()
    print(label)

run_t0_dep2('0,0,0',[('a','p'),('N1pi1,a','12'),],ylim2=[0,0.5],ylim3=[0,0.05],ylim4=[0,0.4],ylim5=[0,0.2],label='_0_N1pi1',choice='eV_0')
run_t0_dep2('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),],ylim2=[0,0.1],ylim3=[0,0.025],ylim4=[0,0.07],ylim5=[0,0.04],label='_1_N1pi0_N0pi1',choice='eV_1')

_0_N1pi1
_1_N1pi0_N0pi1


In [10]:
# with choices

flags['dt2pt']=False
flags['dt3pt']=True

tfs=[10,12,14]
def get_dt(eVecsa_ltni,eVecsb_ltni):
    eVecsa_ltni=yu.renormalize_eVecs(eVecsa_ltni)
    # eVecsa_ltni=eVecsa_ltni/np.diagonal(eVecsa_ltni,axis1=-2,axis2=-1)[:,:,:,None]
    eVecsa_lni=np.mean(eVecsa_ltni,axis=1)
    eVecsa_inv_lin=np.linalg.inv(eVecsa_lni)
    Wa_l=1/(eVecsa_lni[:,0,0]*eVecsa_inv_lin[:,0,0])-1
    eVeca_li=eVecsa_lni[:,0]
    dt2pta_lij=np.einsum('li,lj->lij',np.conj(eVeca_li),eVeca_li)
    if flags['dt2pt']:
        dt2pta_lij[:,1:,1:]*=0
        dt2pta_lij[:,0,0]*=1-np.conj(Wa_l)*Wa_l
        dt2pta_lij[:,1:,0]*=1+Wa_l[:,None]
        dt2pta_lij[:,0,1:]*=1+np.conj(Wa_l[:,None])

    eVecsb_ltni=yu.renormalize_eVecs(eVecsb_ltni)
    # eVecsb_ltni=eVecsb_ltni/np.diagonal(eVecsb_ltni,axis1=-2,axis2=-1)[:,:,:,None]
    eVecsb_lni=np.mean(eVecsb_ltni,axis=1)
    eVecsb_inv_lin=np.linalg.inv(eVecsb_lni)
    Wb_l=1/(eVecsb_lni[:,0,0]*eVecsb_inv_lin[:,0,0])-1
    eVecb_li=eVecsb_lni[:,0]
    dt2ptb_lij=np.einsum('li,lj->lij',np.conj(eVecb_li),eVecb_li)
    if flags['dt2pt']:
        dt2ptb_lij[:,1:,1:]*=0
        dt2ptb_lij[:,0,0]*=1-np.conj(Wb_l)*Wb_l
        dt2ptb_lij[:,1:,0]*=1+Wb_l[:,None]
        dt2ptb_lij[:,0,1:]*=1+np.conj(Wb_l[:,None])

    dt3pt_ijlm=np.einsum('li,mj->ijlm',np.conj(eVeca_li),eVecb_li)
    dt3pt_ijlm[1:,1:]*=0
    if flags['dt3pt']:
        d00_lm=1-np.einsum('l,m->lm',np.conj(Wa_l),Wb_l)
        di0_lm=1+np.einsum('l,m->lm',[1,1],Wb_l)
        d0j_lm=1+np.einsum('l,m->lm',np.conj(Wa_l),[1,1])
        dt3pt_ijlm[1:,1:]*=0
        dt3pt_ijlm[0,0]*=d00_lm
        dt3pt_ijlm[1:,0]*=di0_lm[None,:,:]
        dt3pt_ijlm[0,1:]*=d0j_lm[None,:,:]

    return (dt2pta_lij,dt2ptb_lij,dt3pt_ijlm)

def runFF(ptofas,ptofbs,FF,choicea,choiceb,label='',ylim=None):
    fig, axs = yu.getFigAxs(4,1,sharex=True,sharey=True)
    yu.addRowHeader(axs,['cA24: noJL','cA24: all','cA48: noJL','cA48: all'])
    # yu.addColHeader(axs,['tRef={}'.format(tRef) for tRef in tRefs])
    # title=title if title=='None' else 'opa={}; opb={}; insert={}; prefactor={}'.format(opa,opb,insert,prefactor)
    # fig.suptitle(title)
    
    for i_ens,ens in enumerate(ensembles):
        updateEns(ens)
        
        t0Lista=[t0 for t0,t in choices[choicea][i_ens]]
        tLista=[t for t0,t in choices[choicea][i_ens]]
        t0Listb=[t0 for t0,t in choices[choiceb][i_ens]]
        tListb=[t for t0,t in choices[choiceb][i_ens]]

        mN=lat.getaEN(0); EN=lat.getaEN(1); qk=(2*math.pi)/(lat.L/lat.a); K=np.sqrt(2*mN**2/(EN*(EN+mN)))
        t_Fpi=92.9/lat.aInv
        t_aQ2=(qk**2-(EN-mN)**2)
        t_P=(lat.ampi**2+t_aQ2)/(92.9/lat.aInv*lat.ampi**2)*(t_aQ2)/(4*mN)
        t_Pt=(2*mN**2)/(K*qk*(mN-EN))*t_P; t_Pt2=(2*mN)/(mN-EN)*t_P
        t_Pi=-1j*(2*mN**2)/(K*qk**2)*t_P; t_Pi2=-(2*mN*(EN+mN))/(qk**2)*t_P
        
        t_A2pNN_0=mN/t_Fpi
        t_A2pNN_1=mN/t_Fpi*(lat.ampi**2+t_aQ2)/(lat.ampi**2)
        t_52pNN_1=(lat.amu/lat.ZP)*(lat.ampi**2+t_aQ2)/(lat.ampi**2)/t_Fpi
        t_P2pNN_1=(t_aQ2)/(4*mN**2)*t_A2pNN_1
        
        t_GAzz_00=(-1)*(2*mN*(mN+mN))/(2j*((mN+mN)**2)) * lat.ZA; t_GAzzb_00=t_GAzz_00 * t_A2pNN_0
        t_GAtz_11=(-1)*(2*EN*(EN+mN))/(-4*(EN+mN)*qk) * lat.ZA; t_GAtzb_11=t_GAtz_11 * t_A2pNN_0
        t_GAzz_11=(-1)*(2*EN*(EN+mN))/(2j*((EN+mN)**2+qk**2)) * lat.ZA; t_GAzzb_11=t_GAzz_11 * t_A2pNN_0
        t_GAxx_11=(-1)*(2*EN*(EN+mN))/(2j*((EN+mN)**2)) * lat.ZA; t_GAxxb_11=t_GAxx_11 * t_A2pNN_0
        
        t_GA_01=(-1)*(-1j)*np.sqrt(EN/(2*(EN+mN))) * lat.ZA
        t_G5_01=(-1*np.sqrt(EN*(EN+mN)/2))/qk * lat.ZS; t_G5b_01=t_G5_01 * t_52pNN_1
        t_GP_01_tz=(-1)*np.sqrt(2*EN*(EN+mN))*mN/(qk*(EN-mN)) * lat.ZA; t_GP_01_tz_A=-(2*mN)/(EN-mN)
        t_GP_01_zz=(-1)*1j*np.sqrt(2*EN*(EN+mN))*mN/(qk**2) * lat.ZA; t_GP_01_zz_A=-(2*mN)/(EN-mN)

        t_PCAC_G5=(lat.amu/lat.ZP)/mN
        t_PCAC_GP=t_aQ2/(4*mN**2)

        xUnit=lat.a
        
        pips={
            'GS+_00':[(lat.ZP,'id_j+','P0')],
            'GA_00_zz':[(t_GAzz_00,'g5gz_j-','Pz')],
            'J5_11':[(lat.ZS,'g5_j-','Pz')],
            'GA_11_tz':[(t_GAtz_11,'g5gt_j-','Pz')],
            'GA_11_zz':[(t_GAzz_11,'g5gz_j-','Pz')],
            'GA_11_xx':[(t_GAxx_11*(1/2),'g5gx_j-','Px'),(t_GAxx_11*(1/2),'g5gy_j-','Py')],
            
            'G5b_01':[(t_G5b_01,'g5_j-','Pz')],
            'GA_01':[(t_GA_01*(1/2),'g5gx_j-','Px'),(t_GA_01*(1/2),'g5gy_j-','Py')],
            'GPb_01_tz':[(t_GA_01*(-1/2)*t_GP_01_tz_A*t_P2pNN_1,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_P2pNN_1,'g5gy_j-','Py'),(t_GP_01_tz*t_P2pNN_1,'g5gt_j-','Pz')],
            'GPb_01_zz':[(t_GA_01*(-1/2)*t_GP_01_zz_A*t_P2pNN_1,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_P2pNN_1,'g5gy_j-','Py'),(t_GP_01_zz*t_P2pNN_1,'g5gz_j-','Pz')],
            
            'PCAC_1':[((-1)*(-(EN-mN)),'g5gt_j-','Pz'),((-1)*(1j*qk),'g5gz_j-','Pz'),((-1)*2*lat.amu,'g5_j-','Pz')],
            'PCAC_1_re':[((-1)*(-(EN-mN))*lat.ZA,'g5gt_j-','Pz'),((-1)*(1j*qk)*lat.ZA,'g5gz_j-','Pz'),((-1)*2*lat.amu/lat.ZP*lat.ZS,'g5_j-','Pz')],
            
            'rPCAC_up_1_Ptz':[(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_tz*t_PCAC_GP,'g5gt_j-','Pz')],
            'rPCAC_up_1_Pzz':[(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_zz*t_PCAC_GP,'g5gz_j-','Pz')],
        
            'rPCAC_1_Ptz':[(t_GA_01*(1/2),'g5gx_j-','Px'),(t_GA_01*(1/2),'g5gy_j-','Py'),(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_tz*t_PCAC_GP,'g5gt_j-','Pz')],
            'rPCAC_1_Pzz':[(t_GA_01*(1/2),'g5gx_j-','Px'),(t_GA_01*(1/2),'g5gy_j-','Py'),(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_zz*t_PCAC_GP,'g5gz_j-','Pz')],
            
            # 'rPCAC2_1':[((-1)*(-(EN-mN))*lat.ZA,'g5gt_j-','Pz'),((-1)*(1j*qk)*lat.ZA,'g5gz_j-','Pz'),(2*lat.amu/lat.ZP*lat.ZS,'g5_j-','Pz')],
        }[FF]
        
        pta,ofas=ptofas; ptb,ofbs=ptofbs

        proj2mat={
            'P0':np.array([[1,0],[0,1]]),
            'Px':np.array([[0,1],[1,0]]),
            'Py':np.array([[0,-1j],[1j,0]]),
            'Pz':np.array([[1,0],[0,-1]]),
        }
        for i_diags,diags in [(0,yu.diags_jLoopless),(1,yu.diags_all)]:
            irow=i_ens*2+i_diags
            for icol,tRef in enumerate([7]):
                needsVEV= pta==ptb and 'N-j' in diags and FF in ['GS+_00','J5_11']
                for case in ['noGEVP','GEVP']:
                    pta,ofas=ptofas; ptb,ofbs=ptofbs
                    if case=='noGEVP':
                        ofas=[ofas[0]]; ofbs=[ofbs[0]]
                    
                    def t_get(tf,t_insert):
                        t=[[get3ptMat(yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs),t_insert+'_'+str(tf),diags)[:,1:tf]\
                            for lb in ['l1','l2']] for la in ['l1','l2']]
                        return np.transpose(t,[2,3,4,5,0,1]) # cfg,tc,opa,opb,la,lb
                    
                    dat3pt=[t_get(tf,insert) for prefactor,insert,proj in pips for tf in tfs]
                    dat2pt=[get2ptMat(yu.getops(pta,'l1',ofas),yu.diags_all),get2ptMat(yu.getops(pta,'l2',ofas),diags),
                            get2ptMat(yu.getops(ptb,'l1',ofbs),yu.diags_all),get2ptMat(yu.getops(ptb,'l2',ofbs),diags),]
                    datVEVj=[data['VEV']['j']['id_j+' if FF in ['GS+_00'] else 'g5_j-']]
                    datVEVpi0f=[data['VEV']['pi0f']]
                    dat=[dat3pt,dat2pt,datVEVj,datVEVpi0f]
                    
                    AInv=yu.getCoeMat(pta); BInv=yu.getCoeMat(ptb)

                    def func(dat):
                        t=yu.meanDeep(dat)
                        t3pt,t2pt,tVEVj,datVEVpi0f=t
                        
                        t2pt[0]=remove_pi0VEV(t2pt[0],datVEVpi0f[0],yu.getops(pta,'l1',ofas),yu.getops(pta,'l1',ofas))
                        t2pt[1]=remove_pi0VEV(t2pt[1],datVEVpi0f[0],yu.getops(pta,'l2',ofas),yu.getops(pta,'l2',ofas))
                        t2pt[2]=remove_pi0VEV(t2pt[2],datVEVpi0f[0],yu.getops(ptb,'l1',ofbs),yu.getops(ptb,'l1',ofbs))
                        t2pt[3]=remove_pi0VEV(t2pt[3],datVEVpi0f[0],yu.getops(ptb,'l2',ofbs),yu.getops(ptb,'l2',ofbs))
                        
                        for ind in range(len(t3pt)):
                            for ia,la in enumerate(['l1','l2']):
                                for ib,lb in enumerate(['l1','l2']): 
                                    t3pt[ind][:,:,:,ia,ib]=remove_pi0VEV(t3pt[ind][:,:,:,ia,ib],datVEVpi0f[0],yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs))
                        
                        eVals_al1,_=yu.GEVP(t2pt[0],1)
                        eVals_al2,_=yu.GEVP(t2pt[1],1)
                        eVals_a=(eVals_al1+eVals_al2)/2
                        Ea_tn=np.log(eVals_a/np.roll(eVals_a,-1,axis=0))
                        
                        eVals_bl1,_=yu.GEVP(t2pt[2],1)
                        eVals_bl2,_=yu.GEVP(t2pt[3],1)
                        eVals_b=(eVals_bl1+eVals_bl2)/2
                        Eb_tn=np.log(eVals_b/np.roll(eVals_b,-1,axis=0))
                        
                        _,eVecs_al1=yu.GEVP(t2pt[0],t0Lista,tList=tLista)
                        _,eVecs_al2=yu.GEVP(t2pt[1],t0Lista,tList=tLista)
                        _,eVecs_bl1=yu.GEVP(t2pt[2],t0Listb,tList=tListb)
                        _,eVecs_bl2=yu.GEVP(t2pt[3],t0Listb,tList=tListb)

                        eVecsa_ltni=np.array([eVecs_al1,eVecs_al2])
                        eVecsb_ltni=np.array([eVecs_bl1,eVecs_bl2])
                        dt2pta_lij,dt2ptb_lij,dt3pt_ijlm=get_dt(eVecsa_ltni,eVecsb_ltni)
                        
                        t2pta_ltij=np.array([t2pt[0],t2pt[1]])
                        t2pta_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2pta_ltij,dt2pta_lij)),axis=0)
                        
                        t2ptb_ltij=np.array([t2pt[2],t2pt[3]])
                        t2ptb_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2ptb_ltij,dt2ptb_lij)),axis=0)
                        
                        if needsVEV:
                            proj=proj2mat[pips[0][2]]
                            t2pt_sub=np.einsum('tij,ij->t',t2pta_ltij[0],dt3pt_ijlm[:,:,0,0])*proj[0,0]+\
                                np.einsum('tij,ij->t',t2pta_ltij[1],dt3pt_ijlm[:,:,1,1])*proj[1,1]
                        
                        if FF not in ['rPCAC_1_Ptz','rPCAC_1_Pzz','rPCAC2_1']:
                            def t_get2(i):
                                res=0
                                for i_pip,pip in enumerate(pips):
                                    prefactor,insert,proj=pip
                                    proj=proj2mat[proj]
                                    proj2=np.einsum('ba,al,bm->lm',proj,AInv,np.conj(BInv))
                                    res+=prefactor*np.einsum('tijlm,lm,ijlm->t',t3pt[i_pip*len(tfs)+i],proj2,dt3pt_ijlm) # ij for ops; l,m for la,lb; ab for diracs
                                return res
                            t=[(
                                t_get2(i)
                                -(t2pt_sub[tf]*tVEVj[0]*pips[0][0] if needsVEV else 0)
                            )/np.sqrt(
                                t2pta_t[tf]*t2ptb_t[tf]*\
                                t2pta_t[1:tf][::-1]/t2pta_t[1:tf]*\
                                t2ptb_t[1:tf]/t2ptb_t[1:tf][::-1]
                                # np.exp(-(Ea_tn[tf,0]-Eb_tn[tf,0])*np.array([tf-2*tc for tc in range(1,tf)]))
                            )
                            for i,tf in enumerate(tfs)]
                        else:
                            def t_get2_N(i):
                                res=0
                                for i_pip,pip in enumerate(pips):
                                    if i_pip<2:
                                        continue
                                    prefactor,insert,proj=pip
                                    proj=proj2mat[proj]
                                    proj2=np.einsum('ba,al,bm->lm',proj,AInv,np.conj(BInv))
                                    res+=prefactor*np.einsum('tijlm,lm,ijlm->t',t3pt[i_pip*len(tfs)+i],proj2,dt3pt_ijlm) # ij for ops; l,m for la,lb; ab for diracs
                                return res
                            def t_get2_D(i):
                                res=0
                                for i_pip,pip in enumerate(pips):
                                    if i_pip>=2:
                                        continue
                                    prefactor,insert,proj=pip
                                    proj=proj2mat[proj]
                                    proj2=np.einsum('ba,al,bm->lm',proj,AInv,np.conj(BInv))
                                    res+=prefactor*np.einsum('tijlm,lm,ijlm->t',t3pt[i_pip*len(tfs)+i],proj2,dt3pt_ijlm) # ij for ops; l,m for la,lb; ab for diracs
                                return res

                            t=[(
                                t_get2_N(i)/t_get2_D(i)
                            )
                            for i,tf in enumerate(tfs)]
                    
                        t=yu.prefactorDeep(t,1)
                        return t
                    
                    (mean,err,cov)=yu.jackknife(dat,func)
                    for i_tf,tf in enumerate(tfs):
                        tMean=mean[i_tf];tErr=err[i_tf]
                        axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)*xUnit,tMean,tErr,color='b' if case=='noGEVP' else 'r',fmt=['s','d','o'][i_tf],mfc='white' if case=='noGEVP' else None)                               
                    if ylim!=None:
                        axs[irow,icol].set_ylim(ylim)
    
    plt.tight_layout()
    plt.savefig('fig/'+FF+label+'.pdf')
    plt.close()
    print(FF)

# for FF in ['G5b_01']:
#     runFF(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),FF,'eV_0','eV_1')

# for FF in ['PCAC_1','PCAC_1_re']:
#     runFF(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),FF,'eV_0','eV_1')

for FF in ['GS+_00','GA_00_zz']:
    runFF(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,0',[('a','p'),('N1pi1,a','12')]),FF,'eV_0','eV_0')
for FF in ['J5_11','GA_11_tz','GA_11_zz','GA_11_xx']:
    runFF(('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),FF,'eV_1','eV_1')
for FF in ['G5b_01','GA_01','GPb_01_tz','GPb_01_zz']:
    runFF(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),FF,'eV_0','eV_1')
for FF in ['rPCAC_1_Ptz','rPCAC_1_Pzz']:
    runFF(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),FF,'eV_0','eV_1')

GS+_00
GA_00_zz
J5_11
GA_11_tz
GA_11_zz
GA_11_xx
G5b_01
GA_01
GPb_01_tz
GPb_01_zz
rPCAC_1_Ptz
rPCAC_1_Pzz


In [ ]:
# disc piNJpiN

tfs=[10,12,14]
def get_dt(eVecsa_ltni,eVecsb_ltni):
    eVecsa_ltni=yu.renormalize_eVecs(eVecsa_ltni)
    # eVecsa_ltni=eVecsa_ltni/np.diagonal(eVecsa_ltni,axis1=-2,axis2=-1)[:,:,:,None]
    eVecsa_lni=np.mean(eVecsa_ltni[:,10:14+1],axis=1)
    eVecsa_inv_lin=np.linalg.inv(eVecsa_lni)
    Wa_l=1/(eVecsa_lni[:,0,0]*eVecsa_inv_lin[:,0,0])-1
    eVeca_li=eVecsa_lni[:,0]
    dt2pta_lij=np.einsum('li,lj->lij',np.conj(eVeca_li),eVeca_li)

    eVecsb_ltni=yu.renormalize_eVecs(eVecsb_ltni)
    # eVecsb_ltni=eVecsb_ltni/np.diagonal(eVecsb_ltni,axis1=-2,axis2=-1)[:,:,:,None]
    eVecsb_lni=np.mean(eVecsb_ltni[:,10:14+1],axis=1)
    eVecsb_inv_lin=np.linalg.inv(eVecsb_lni)
    Wb_l=1/(eVecsb_lni[:,0,0]*eVecsb_inv_lin[:,0,0])-1
    eVecb_li=eVecsb_lni[:,0]
    dt2ptb_lij=np.einsum('li,lj->lij',np.conj(eVecb_li),eVecb_li)
    
    dt3pt_ijlm=np.einsum('li,mj->ijlm',np.conj(eVeca_li),eVecb_li)
    return (dt2pta_lij,dt2ptb_lij,dt3pt_ijlm)

def runFF(ptofas,ptofbs,FF,tRefs=[1],label='',ylim1=None):
    fig, axs = yu.getFigAxs(6,len(tRefs),sharex=True,sharey=True if ylim1==None else False)
    yu.addRowHeader(axs,['cA24: noJL','cA24: all', 'cA24: extend','cA48: noJL','cA48: all','cA48: extend'])
    yu.addColHeader(axs,['tRef={}'.format(tRef) for tRef in tRefs])
    # title=title if title=='None' else 'opa={}; opb={}; insert={}; prefactor={}'.format(opa,opb,insert,prefactor)
    # fig.suptitle(title)
    
    for i_ens,ens in enumerate(ensembles):
        updateEns(ens)

        mN=lat.getaEN(0); EN=lat.getaEN(1); qk=(2*math.pi)/(lat.L/lat.a); K=np.sqrt(2*mN**2/(EN*(EN+mN)))
        t_Fpi=92.9/lat.aInv
        t_aQ2=(qk**2-(EN-mN)**2)
        t_P=(lat.ampi**2+t_aQ2)/(92.9/lat.aInv*lat.ampi**2)*(t_aQ2)/(4*mN)
        t_Pt=(2*mN**2)/(K*qk*(mN-EN))*t_P; t_Pt2=(2*mN)/(mN-EN)*t_P
        t_Pi=-1j*(2*mN**2)/(K*qk**2)*t_P; t_Pi2=-(2*mN*(EN+mN))/(qk**2)*t_P
        
        t_A2pNN_0=mN/t_Fpi
        t_A2pNN_1=mN/t_Fpi*(lat.ampi**2+t_aQ2)/(lat.ampi**2)
        t_52pNN_1=(lat.amu/lat.ZP)*(lat.ampi**2+t_aQ2)/(lat.ampi**2)/t_Fpi
        t_P2pNN_1=(t_aQ2)/(4*mN**2)*t_A2pNN_1
        
        t_GAzz_00=(-1)*(2*mN*(mN+mN))/(2j*((mN+mN)**2)) * lat.ZA; t_GAzzb_00=t_GAzz_00 * t_A2pNN_0
        t_GAtz_11=(-1)*(2*EN*(EN+mN))/(-4*(EN+mN)*qk) * lat.ZA; t_GAtzb_11=t_GAtz_11 * t_A2pNN_0
        t_GAzz_11=(-1)*(2*EN*(EN+mN))/(2j*((EN+mN)**2+qk**2)) * lat.ZA; t_GAzzb_11=t_GAzz_11 * t_A2pNN_0
        t_GAxx_11=(-1)*(2*EN*(EN+mN))/(2j*((EN+mN)**2)) * lat.ZA; t_GAxxb_11=t_GAxx_11 * t_A2pNN_0
        
        t_GA_01=(-1)*(-1j)*np.sqrt(EN/(2*(EN+mN))) * lat.ZA
        t_G5_01=(-1*np.sqrt(EN*(EN+mN)/2))/qk * lat.ZS; t_G5b_01=t_G5_01 * t_52pNN_1
        t_GP_01_tz=(-1)*np.sqrt(2*EN*(EN+mN))*mN/(qk*(EN-mN)) * lat.ZA; t_GP_01_tz_A=-(2*mN)/(EN-mN)
        t_GP_01_zz=(-1)*1j*np.sqrt(2*EN*(EN+mN))*mN/(qk**2) * lat.ZA; t_GP_01_zz_A=-(2*mN)/(EN-mN)

        t_PCAC_G5=(lat.amu/lat.ZP)/mN
        t_PCAC_GP=t_aQ2/(4*mN**2)

        xUnit=lat.a
        
        pips={
            'GS+_00':[(lat.ZP,'id_j+','P0')],
            'GA_00_zz':[(t_GAzz_00,'g5gz_j-','Pz')],
            'J5_11':[(lat.ZS,'g5_j-','Pz')],
            'GA_11_tz':[(t_GAtz_11,'g5gt_j-','Pz')],
            'GA_11_zz':[(t_GAzz_11,'g5gz_j-','Pz')],
            'GA_11_xx':[(t_GAxx_11*(1/2),'g5gx_j-','Px'),(t_GAxx_11*(1/2),'g5gy_j-','Py')],
            
            'G5b_01':[(t_G5b_01,'g5_j-','Pz')],
            'GA_01':[(t_GA_01*(1/2),'g5gx_j-','Px'),(t_GA_01*(1/2),'g5gy_j-','Py')],
            'GPb_01_tz':[(t_GA_01*(-1/2)*t_GP_01_tz_A*t_P2pNN_1,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_P2pNN_1,'g5gy_j-','Py'),(t_GP_01_tz*t_P2pNN_1,'g5gt_j-','Pz')],
            'GPb_01_zz':[(t_GA_01*(-1/2)*t_GP_01_zz_A*t_P2pNN_1,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_P2pNN_1,'g5gy_j-','Py'),(t_GP_01_zz*t_P2pNN_1,'g5gz_j-','Pz')],
            
            'PCAC_1':[((-1)*(-(EN-mN)),'g5gt_j-','Pz'),((-1)*(1j*qk),'g5gz_j-','Pz'),((-1)*2*lat.amu,'g5_j-','Pz')],
            'PCAC_1_re':[((-1)*(-(EN-mN))*lat.ZA,'g5gt_j-','Pz'),((-1)*(1j*qk)*lat.ZA,'g5gz_j-','Pz'),((-1)*2*lat.amu/lat.ZP*lat.ZS,'g5_j-','Pz')],
            
            'rPCAC_up_1_Ptz':[(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_tz*t_PCAC_GP,'g5gt_j-','Pz')],
            'rPCAC_up_1_Pzz':[(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_zz*t_PCAC_GP,'g5gz_j-','Pz')],
        
            'rPCAC_1_Ptz':[(t_GA_01*(1/2),'g5gx_j-','Px'),(t_GA_01*(1/2),'g5gy_j-','Py'),(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_tz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_tz*t_PCAC_GP,'g5gt_j-','Pz')],
            'rPCAC_1_Pzz':[(t_GA_01*(1/2),'g5gx_j-','Px'),(t_GA_01*(1/2),'g5gy_j-','Py'),(t_G5_01*t_PCAC_G5,'g5_j-','Pz'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gx_j-','Px'),(t_GA_01*(-1/2)*t_GP_01_zz_A*t_PCAC_GP,'g5gy_j-','Py'),(t_GP_01_zz*t_PCAC_GP,'g5gz_j-','Pz')],
        }[FF]
        
        pta,ofas=ptofas; ptb,ofbs=ptofbs

        proj2mat={
            'P0':np.array([[1,0],[0,1]]),
            'Px':np.array([[0,1],[1,0]]),
            'Py':np.array([[0,-1j],[1j,0]]),
            'Pz':np.array([[1,0],[0,-1]]),
        }
        for i_diags,diags in [(0,yu.diags_jLoopless),(1,yu.diags_all)]:
            irow=i_ens*3+i_diags
            for icol,tRef in enumerate(tRefs):
                needsVEV= pta==ptb and 'N-j' in diags and FF in ['GS+_00','J5_11']
                for case in ['noGEVP','GEVP']:
                    pta,ofas=ptofas; ptb,ofbs=ptofbs
                    if case=='noGEVP':
                        ofas=[ofas[0]]; ofbs=[ofbs[0]]
                    
                    def t_get(tf,t_insert):
                        t=[[get3ptMat(yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs),t_insert+'_'+str(tf),diags)[:,1:tf]\
                            for lb in ['l1','l2']] for la in ['l1','l2']]
                        return np.transpose(t,[2,3,4,5,0,1]) # cfg,tc,opa,opb,la,lb
                    
                    dat3pt=[t_get(tf,insert) for prefactor,insert,proj in pips for tf in tfs]
                    dat2pt=[get2ptMat(yu.getops(pta,'l1',ofas),yu.diags_all),get2ptMat(yu.getops(pta,'l2',ofas),diags),
                            get2ptMat(yu.getops(ptb,'l1',ofbs),yu.diags_all),get2ptMat(yu.getops(ptb,'l2',ofbs),diags),]
                    datVEVj=[data['VEV']['j']['id_j+' if FF in ['GS+_00'] else 'g5_j-']]
                    datVEVpi0f=[data['VEV']['pi0f']]
                    dat=[dat3pt,dat2pt,datVEVj,datVEVpi0f]
                    
                    AInv=yu.getCoeMat(pta); BInv=yu.getCoeMat(ptb)
                    
                    flag_piNJpiN=False

                    def func(dat):
                        t=yu.meanDeep(dat)
                        t3pt,t2pt,tVEVj,datVEVpi0f=t
                        
                        t2pt[0]=remove_pi0VEV(t2pt[0],datVEVpi0f[0],yu.getops(pta,'l1',ofas),yu.getops(pta,'l1',ofas))
                        t2pt[1]=remove_pi0VEV(t2pt[1],datVEVpi0f[0],yu.getops(pta,'l2',ofas),yu.getops(pta,'l2',ofas))
                        t2pt[2]=remove_pi0VEV(t2pt[2],datVEVpi0f[0],yu.getops(ptb,'l1',ofbs),yu.getops(ptb,'l1',ofbs))
                        t2pt[3]=remove_pi0VEV(t2pt[3],datVEVpi0f[0],yu.getops(ptb,'l2',ofbs),yu.getops(ptb,'l2',ofbs))
                        
                        for ind in range(len(t3pt)):
                            for ia,la in enumerate(['l1','l2']):
                                for ib,lb in enumerate(['l1','l2']): 
                                    t3pt[ind][:,:,:,ia,ib]=remove_pi0VEV(t3pt[ind][:,:,:,ia,ib],datVEVpi0f[0],yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs))
                        
                        (eVals_al1,eVecs_al1)=yu.GEVP(t2pt[0],tRef)
                        (eVals_al2,eVecs_al2)=yu.GEVP(t2pt[1],tRef)
                        (eVals_bl1,eVecs_bl1)=yu.GEVP(t2pt[2],tRef)
                        (eVals_bl2,eVecs_bl2)=yu.GEVP(t2pt[3],tRef)
                        
                        eVals_a=(eVals_al1+eVals_al2)/2
                        Ea_tn=np.log(eVals_a/np.roll(eVals_a,-1,axis=0))
                        eVals_b=(eVals_bl1+eVals_bl2)/2
                        Eb_tn=np.log(eVals_b/np.roll(eVals_b,-1,axis=0))
                        
                        eVecsa_ltni=np.array([eVecs_al1,eVecs_al2])
                        eVecsb_ltni=np.array([eVecs_bl1,eVecs_bl2])
                        dt2pta_lij,dt2ptb_lij,dt3pt_ijlm=get_dt(eVecsa_ltni,eVecsb_ltni)
                        
                        t2pta_ltij=np.array([t2pt[0],t2pt[1]])
                        t2pta_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2pta_ltij,dt2pta_lij)),axis=0)
                        
                        t2ptb_ltij=np.array([t2pt[2],t2pt[3]])
                        t2ptb_t=np.mean(np.real(np.einsum('ltij,lij->lt',t2ptb_ltij,dt2ptb_lij)),axis=0)
                        
                        if not flag_piNJpiN:
                            dt3pt_ijlm[1:,1:]*=0
                        
                        if needsVEV:
                            proj=proj2mat[pips[0][2]]
                            t2pt_sub=np.einsum('tij,ij->t',t2pta_ltij[0],dt3pt_ijlm[:,:,0,0])*proj[0,0]+\
                                np.einsum('tij,ij->t',t2pta_ltij[1],dt3pt_ijlm[:,:,1,1])*proj[1,1]
                        
                        if FF not in ['rPCAC_1_Ptz','rPCAC_1_Pzz']:
                            def t_get2(i):
                                res=0
                                for i_pip,pip in enumerate(pips):
                                    prefactor,insert,proj=pip
                                    proj=proj2mat[proj]
                                    proj2=np.einsum('ba,al,bm->lm',proj,AInv,np.conj(BInv))
                                    res+=prefactor*np.einsum('tijlm,lm,ijlm->t',t3pt[i_pip*len(tfs)+i],proj2,dt3pt_ijlm) # ij for ops; l,m for la,lb; ab for diracs
                                return res
                            t=[(
                                t_get2(i)
                                -(t2pt_sub[tf]*tVEVj[0]*pips[0][0] if needsVEV else 0)
                            )/np.sqrt(
                                t2pta_t[tf]*t2ptb_t[tf]*\
                                # t2pta_t[1:tf][::-1]/t2pta_t[1:tf]*\
                                # t2ptb_t[1:tf]/t2ptb_t[1:tf][::-1]
                                np.exp(-(Ea_tn[tf,0]-Eb_tn[tf,0])*np.array([tf-2*tc for tc in range(1,tf)]))
                            )
                            for i,tf in enumerate(tfs)]
                        else:
                            def t_get2_N(i):
                                res=0
                                for i_pip,pip in enumerate(pips):
                                    if i_pip<2:
                                        continue
                                    prefactor,insert,proj=pip
                                    proj=proj2mat[proj]
                                    proj2=np.einsum('ba,al,bm->lm',proj,AInv,np.conj(BInv))
                                    res+=prefactor*np.einsum('tijlm,lm,ijlm->t',t3pt[i_pip*len(tfs)+i],proj2,dt3pt_ijlm) # ij for ops; l,m for la,lb; ab for diracs
                                return res
                            def t_get2_D(i):
                                res=0
                                for i_pip,pip in enumerate(pips):
                                    if i_pip>=2:
                                        continue
                                    prefactor,insert,proj=pip
                                    proj=proj2mat[proj]
                                    proj2=np.einsum('ba,al,bm->lm',proj,AInv,np.conj(BInv))
                                    res+=prefactor*np.einsum('tijlm,lm,ijlm->t',t3pt[i_pip*len(tfs)+i],proj2,dt3pt_ijlm) # ij for ops; l,m for la,lb; ab for diracs
                                return res

                            t=[(
                                t_get2_N(i)/t_get2_D(i)
                            )
                            for i,tf in enumerate(tfs)]
                    
                        t=yu.prefactorDeep(t,1)
                        return t
                    
                    (mean,err,cov)=yu.jackknife(dat,func)
                    for i_tf,tf in enumerate(tfs):
                        tMean=mean[i_tf];tErr=err[i_tf]
                        axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)*xUnit,tMean,tErr,color='b' if case=='noGEVP' else 'r',fmt=['s','d','o'][i_tf],mfc='white' if case=='noGEVP' else None)                                         

                    if i_ens==0 or i_diags==0:
                        continue
                    flag_piNJpiN=True
                    (mean,err,cov)=yu.jackknife(dat,func)
                    for i_tf,tf in enumerate(tfs):
                        tMean=mean[i_tf];tErr=err[i_tf]
                        axs[irow+1,icol].errorbar(np.arange(1 - tf//2,tf//2)*xUnit,tMean,tErr,color='b' if case=='noGEVP' else 'r',fmt=['s','d','o'][i_tf],mfc='white' if case=='noGEVP' else None)         
    
    plt.tight_layout()
    plt.savefig('fig/'+FF+label+'.pdf')
    plt.close()
    print(FF)
    
# tRefs=[1,7]
# for FF in ['J5_11']:
#     runFF(('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')]),FF,tRefs=tRefs,label='_')

# tRefs=[1,2,3,4,5,6,7]
# for FF in ['GS+_00','GA_00_zz']:
#     runFF(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,0',[('a','p'),('N1pi1,a','12')]),FF,tRefs=tRefs)
# for FF in ['J5_11','GA_11_tz','GA_11_zz','GA_11_xx']:
#     runFF(('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),FF,tRefs=tRefs)
# for FF in ['G5b_01','GA_01','GPb_01_tz','GPb_01_zz']:
#     runFF(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),FF,tRefs=tRefs)
# for FF in ['rPCAC_1_Ptz','rPCAC_1_Pzz']:
#     runFF(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),FF,tRefs=tRefs)